In [7]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import os 
import random
import warnings

import torch

warnings.filterwarnings("ignore")

SEED_VALUE = 100
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)
torch.cuda.manual_seed(SEED_VALUE)
torch.cuda.manual_seed_all(SEED_VALUE)


torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# city pollution data

In [50]:
# df = pd.read_csv("./data/city_pollution_data.csv")
df = pd.read_csv("./data/IR2019to2023.csv")


DROP_ONEHOT = True
SEQ_LENGTH = 5


if DROP_ONEHOT:
  INPUT_DIM = 8
else:
  INPUT_DIM = 29

HIDDEN_DIM = 32
LAYER_DIM = 3


normalization_type = 'mean_std' # 'max', mean_std

data pre-processing

In [51]:
import datetime

def get_train_test_data(df):
  # we'll mostly need median and variance values of features for most of our needs

  for col in df.columns:
    for x in ["Country", "min", "max", "count", "County", "past_week", "latitude", "longitude", "State", "variance"]:
      if x in col:
        df.drop([col], axis=1, inplace=True)

  # df["Population Staying at Home"] = df["Population Staying at Home"].apply(lambda x: x.replace(",", ""))
  # df["Population Not Staying at Home"] = df["Population Not Staying at Home"].apply(lambda x: x.replace(",", ""))

  # Now we want 2 more features. Which day of week it is and which month it is.
  # Both of these will be one-hot and hence we'll add 7+12 = 19 more columns.
  # Getting month id is easy from the datetime column. 
  # For day of week, we'll use datetime library.
  
  df['weekday'] = df['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").weekday())
  df['month'] = df['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").month - 1)

  # using one-hot on month and weekday
  weekday_onehot = pd.get_dummies(df['weekday'])
  weekday_onehot.columns = ["day_"+str(x) for x in weekday_onehot]
  month_onehot = pd.get_dummies(df['month'])
  month_onehot.columns = ["month_"+str(x) for x in month_onehot]

  df.drop(['weekday', 'month'], axis=1, inplace=True)
  df = df.join([weekday_onehot, month_onehot])

  cities_list = list(set(df['City']))
  print(cities_list)
  cities_list.sort()
  print(cities_list)
  city_df = {}
  test_indices_of_cities = {}
  train_set = {}
  test_set = {}
  TEST_SET_SIZE = 60                                        

  for city in cities_list:
    city_df[city] = df[df['City'] == city].sort_values('Date').reset_index()
    for col in city_df[city].columns:
      if col in ["median_pm25", "median_o3", "median_so2", "median_no2", "median_pm10", "median_co"]:
        continue
      try:  
        _mean = np.nanmean(city_df[city][col])
        if np.isnan(_mean) == True:
          _mean = 0
        city_df[city][col] = city_df[city][col].fillna(_mean)
      except:
        pass
    if city_df[city].shape[0] < 600 :
      print("City with less than 600 data : {} {}".format(city_df[city].shape[0], city))
      del city_df[city]
      continue
    
    test_index_start = random.randint(0, city_df[city].shape[0] - TEST_SET_SIZE)
    test_indices_of_cities[city] = [test_index_start, test_index_start + TEST_SET_SIZE]

    test_set[city] = city_df[city].iloc[test_index_start:test_index_start + TEST_SET_SIZE]
    train_set[city] = city_df[city].drop(index=list(range(test_index_start, test_index_start + TEST_SET_SIZE)))

  return train_set, test_set

In [52]:
train_set, test_set = get_train_test_data(df)

cities_list = list(train_set.keys())

all_train = pd.DataFrame()
for city in cities_list:
  all_train = all_train.append(train_set[city], ignore_index=True)

all_test = pd.DataFrame({})
for city in test_set:
  all_test = all_test.append(test_set[city], ignore_index=True)

concat_df = pd.concat([all_train,all_test],axis=0)

['Tehran', 'Orūmīyeh', 'Zanjān', 'Kerman', 'Sanandaj', 'Karaj', 'Khorramshahr', 'Khorramabad', 'Arāk', 'Qom', 'Yazd', 'Īlām', 'Mashhad', 'Bandar Abbas', 'Kermanshah', 'Isfahan', 'Tabriz', 'Shiraz']
['Arāk', 'Bandar Abbas', 'Isfahan', 'Karaj', 'Kerman', 'Kermanshah', 'Khorramabad', 'Khorramshahr', 'Mashhad', 'Orūmīyeh', 'Qom', 'Sanandaj', 'Shiraz', 'Tabriz', 'Tehran', 'Yazd', 'Zanjān', 'Īlām']


In [36]:
col_max = {}
col_mean = {}
col_mean2 = {}
col_std = {}

for city in cities_list:
  col_mean[city] = {}
  for col in train_set[city]:
    if col in ["index", "Date", "City"]:
      continue

    train_set[city][col] = train_set[city][col].astype("float")
    test_set[city][col] = test_set[city][col].astype("float")

    if col in ["median_pm25", "median_o3", "median_so2", "median_no2", "median_pm10", "median_co"]:
      _mean = np.nanmean(train_set[city][col])
      if np.isnan(_mean) == True:
        _mean = 0
      
      col_mean[city][col] = _mean
      train_set[city][col] = train_set[city][col].fillna(_mean)

    if normalization_type == 'mean_std':
      col_mean2[col] = np.nanmean(concat_df[col].astype("float"))
      col_std[col] = np.nanstd(concat_df[col].astype("float"))
      train_set[city][col] = (train_set[city][col] - col_mean2[col]) / (col_std[col] + 0.001)
      test_set[city][col] = (test_set[city][col] - col_mean2[col]) / (col_std[col] + 0.001)

    else:
      col_max[col] = concat_df[col].astype("float").max()
      train_set[city][col] = train_set[city][col] / (col_max[col] + 0.001)
      test_set[city][col] = test_set[city][col] / (col_max[col] + 0.001)

  if DROP_ONEHOT:
    train_set[city].drop(train_set[city].columns[-19:], axis=1, inplace=True)
    test_set[city].drop(test_set[city].columns[-19:], axis=1, inplace=True)


In [37]:
# number of data per city

print("num of cities : ", len(cities_list))
for city in cities_list:
    print(city+"({})".format(len(train_set[city])), end=", ")

num of cities :  20
Belfast(1613), Birmingham(1612), Bristol(1612), Cardiff(1612), Coventry(1613), Edinburgh(1613), Glasgow(1612), Leeds(1613), Leicester(1612), Liverpool(1613), London(1612), Manchester(1612), Newport(1612), Norwich(1602), Plymouth(1612), Preston(1612), Reading(1499), Sheffield(1612), Southend-on-Sea(1612), Swansea(1612), 

In [7]:
class CityDataP(torch.utils.data.Dataset):
  def __init__(self, selected_column, split, city_to_use):
    self.split = split
    if split == "train":
      self.dataset = train_set
    else:
      self.dataset = test_set

    self.valid_city_idx = 0
    self.valid_day_idx = 0
    self.selected_column = selected_column
    self.city_to_use = city_to_use
    
    self.city_index = cities_list.index(self.city_to_use)

  def __getitem__(self, idx):
    if self.split != "train":
      # getting all data out of the validation set
      out, city = self.get_idx_data(idx)
      # print(out.columns)
    
    else:
      # getting data randomly for train split
      city = self.city_to_use
      _df = self.dataset[city]
      start_idx = random.randint(1,_df.shape[0]-SEQ_LENGTH)
      out =  _df.iloc[start_idx-1:start_idx+SEQ_LENGTH]

    out = out.drop(['index', 'Date', 'City'], axis=1)

    Y = pd.DataFrame({})
    Y_true = pd.DataFrame({})

    for col in out.columns:
      if col == self.selected_column:
        Y_true[col] = out[col]
        Y[col] = out[col].fillna(col_mean[city][col])
      
      if col in ["median_pm25", "median_o3", "median_so2", "median_no2", "median_pm10", "median_co"]:
        out.drop([col], axis=1, inplace=True)
      else:
        out[col] = out[col].astype("float")
    # print(out.columns) 
    # print(out.columns)
    # print("out shape : ", out.shape)
    # print("Y shape : ", Y.shape)
    out = np.concatenate((np.array(out)[1:,:], np.array(Y)[:-1,:]), axis=1)
    Y = np.array(Y)[1:]
    Y_true = np.array(Y_true)[1:]
    
    return out, Y, Y_true

  def get_idx_data(self, idx):
    # city_idx = self.valid_city_idx % len(cities_list)
    # city = cities_list[city_idx]
    city = self.city_to_use
    _df = self.dataset[city]

    out =  _df.iloc[self.valid_day_idx:self.valid_day_idx+SEQ_LENGTH]
    
    if self.valid_day_idx+SEQ_LENGTH >= _df.shape[0]:
      self.valid_day_idx = 0
      self.valid_city_idx += 1
    else:
      self.valid_day_idx += 1

    return out, city

  def __len__(self):
    if self.split != "train":
      return (61-SEQ_LENGTH)
    return len(train_set[self.city_to_use]) - (SEQ_LENGTH - 1)


In [8]:
class CityDataForecast(torch.utils.data.Dataset):
  def __init__(self, selected_column, split, city_to_use):
    self.split = split
    if split == "train":
      self.dataset = train_set
    else:
      self.dataset = test_set

    self.valid_city_idx = 0
    self.valid_day_idx = 0
    self.selected_column = selected_column
    self.city_to_use = city_to_use
    
    self.city_index = cities_list.index(self.city_to_use)
    

  def __getitem__(self, idx):
    city = self.city_to_use
    _df = self.dataset[city]
    start_idx = random.randint(1,_df.shape[0]-1)
    out =  _df.iloc[start_idx-1:start_idx+1]

    out = out.drop(['index', 'Date', 'City'], axis=1)

    Y = pd.DataFrame({})
    Y_true = pd.DataFrame({})

    for col in out.columns:
      if col == self.selected_column:
        Y_true[col] = out[col]
        #print(out[col])
        Y[col] = out[col].fillna(col_mean[city][col])
      
      if col in ["median_pm25", "median_o3", "median_so2", "median_no2", "median_pm10", "median_co"]:
        out.drop([col], axis=1, inplace=True)
      else:
        out[col] = out[col].astype("float")

    # print(out.shape)
    out = np.concatenate((np.array(out)[1:,:], np.array(Y)[:-1,:]), axis=1)
    Y = np.array(Y)[1:]
    Y_true = np.array(Y_true)[1:]
    

    return out, Y, Y_true

  def get_idx_data(self, idx):
    city = cities_list[self.valid_city_idx]
    _df = self.dataset[city]

    out =  _df.iloc[self.valid_day_idx:self.valid_day_idx+1]
    
    if self.valid_day_idx+1 >= _df.shape[0]:
      self.valid_day_idx = 0
      self.valid_city_idx += 1
    else:
      self.valid_day_idx += 1

    return out, city

  def __len__(self):
    if self.split != "train":
      return (61-1)
    return len(train_set[self.city_to_use])

In [9]:
# function that implement the look_ahead mask for masking future time steps. 
def create_look_ahead_mask(size, device):
    mask = torch.ones((size, size), device=device)
    mask = torch.triu(mask, diagonal=1)
    return mask  # (size, size)

In [10]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

# Model fitting

performance of each linear & nonlinear model

In [11]:
MSEs = {}
RMSEs = {}
MAPEs = {}
models = ['RNN', 'LSTM', 'TransLSTM', 'Transformer', 'CosFormer', 'CosSquareFormer', 'OLS', 'LASSO', 'Ridge', 'Elastic']

for model in models :
    MSEs[model] = []
    RMSEs[model] = []
    MAPEs[model] = []
    

In [12]:
from loss_utils import *
from models import *

from torch import nn
from torch import optim
import torch

device="cuda"


# Evaluation
def evaluation(val_loader, model, model_name, LUR, SELECTED_COLUMN, mask=False):
    model.to(device)
    model.eval()
    mse_list = []
    total_se = 0.0
    total_pe = 0.0
    total_valid = 0.0

    for x_val, _, y_val in val_loader:
        x_val, y_val = [t.cuda().float() for t in (x_val, y_val)]
        
        if mask:
            masking = create_look_ahead_mask(x_val.shape[1], device)
            out, _ = model(x_val.to(device), masking)
        else:
            out = model(x_val.to(device))

        if LUR :
            ytrue = y_val[:,-1,:].squeeze().cpu().numpy()
            ypred = out[:,-1,:].squeeze().cpu().detach().numpy()
        else:
            ytrue = y_val[:,-1,:].squeeze().cpu().numpy()
            ypred = out[:,-1,:].squeeze().cpu().detach().numpy()
        true_valid = np.isnan(ytrue) != 1
        ytrue = ytrue[true_valid] #np.nan_to_num(ytrue, 0)
        ypred = ypred[true_valid]

        if normalization_type == 'mean_std':
            ytrue = (ytrue * col_std[SELECTED_COLUMN]) + col_mean2[SELECTED_COLUMN]
            ypred = (ypred * col_std[SELECTED_COLUMN]) + col_mean2[SELECTED_COLUMN]
        
        else:
            ytrue = (ytrue * col_max[SELECTED_COLUMN])
            ypred = (ypred * col_max[SELECTED_COLUMN])

        se = (ytrue - ypred)**2 # np.square(ytrue - ypred)
        pe = np.abs((ytrue - ypred) / (ytrue + 0.0001))
        total_se += np.sum(se)
        total_pe += np.sum(pe)
        total_valid += np.sum(true_valid)

    eval_mse = total_se / total_valid # np.mean(se) # 
    eval_mape = total_pe / total_valid # np.mean(pe) # 
    print('valid samples:', total_valid)
    print('Eval MSE: ', eval_mse)
    print('Eval RMSE: {}: '.format(SELECTED_COLUMN), np.sqrt(eval_mse))
    print('Eval MAPE: {}: '.format(SELECTED_COLUMN), eval_mape*100)
    
    return eval_mse, eval_mape*100


# Train
def train(sampleLoader, val_loader, model, model_name, SELECTED_COLUMN, mask=False, LUR=False, l1=False, l2=False):

    lr = 0.01
    n_epochs = 10   
    
    model.to(device)

    criterion = nn.MSELoss()
    
    # LUR
    if LUR:
        print("set l1,l2 loss")
        l1_lmbda = 0.01
        l1_lmbda = torch.FloatTensor([l1_lmbda]).cuda()
        l1_reg = torch.tensor(0., requires_grad=True).to(device)
        l2_lmbda = 0.01
        l2_lmbda = torch.FloatTensor([l2_lmbda]).cuda()
        l2_reg = torch.tensor(0., requires_grad=True).to(device)
        opt = torch.optim.SGD(model.parameters(), lr=lr)
            
    # DL
    else:
        print("set SoftDTW loss")
        lmbda = 0.5
        dtw_loss = SoftDTW(use_cuda=True, gamma=0.1)
        opt = torch.optim.SGD(model.parameters(), lr=lr)
    
      
    print('Start ' + model_name + ' training')
    best_mse = 2000.0
    mape = 2000.0
    best_model = None
    

    for epoch in range(1, n_epochs + 1):
        epoch_loss = 0
        batch_idx = 0
        bar = tqdm(sampleLoader)

        model.train()
        for x_batch, y_batch, _ in bar:
            model.train()
            x_batch = x_batch.cuda().float()
            y_batch = y_batch.cuda().float()

            
            if mask==True:
                masking = create_look_ahead_mask(x_batch.shape[1], device)
                out, _ = model(x_batch.to(device), masking)
            else :
                out = model(x_batch.to(device))
            opt.zero_grad()
            
            if LUR:
                # LASSO
                if l1==True and l2==False:
                    l1_reg = torch.norm(model.linear.weight, p=1)
                    loss = criterion(out[:,-1,:], y_batch[:,-1,:]) + l1_lmbda * l1_reg
                # Ridge
                elif l1==False and l2==True:
                    l2_reg = torch.norm(model.linear.weight, p=2)
                    loss = criterion(out[:,-1,:], y_batch[:,-1,:]) + l2_lmbda * l2_reg
                # Elastic
                elif l1==True and l2==True:
                    l1_reg = torch.norm(model.linear.weight, p=1)
                    l2_reg = torch.norm(model.linear.weight, p=2)
                    loss = criterion(out[:,-1,:], y_batch[:,-1,:]) + l1_lmbda * l1_reg + l2_lmbda * l2_reg
                # OLS
                else:
                    loss = criterion(out[:,-1,:], y_batch[:,-1,:])
            else:
                loss = criterion(out[:,-1,:], y_batch[:,-1,:]) + lmbda * dtw_loss(out.cuda(),y_batch.cuda()).mean()

            epoch_loss = (epoch_loss*batch_idx + loss.item())/(batch_idx+1)
            loss.backward(retain_graph=True)
            opt.step()

            bar.set_description(str(epoch_loss))
            batch_idx += 1

        eval_mse, eval_mape = evaluation(val_loader, model, model_name, LUR, SELECTED_COLUMN, mask)
        
        city = sampleLoader.dataset.city_to_use

        if eval_mse < best_mse:
            best_model = deepcopy(model)
            best_mse = eval_mse
            mape = eval_mape
            torch.save(best_model.state_dict(), "./save_US/"+SELECTED_COLUMN+"/"+model_name+"_"+city+".pth")
      
    print(model_name)   
    print("Best MSE :", best_mse)
    print("RMSE :", np.sqrt(best_mse))
    print("MAPE :", mape)
    print()
    MSEs[model_name].append(best_mse)
    RMSEs[model_name].append(np.sqrt(best_mse))
    MAPEs[model_name].append(mape)


In [13]:
SELECTED_COLUMN = "median_no2" # ["median_pm25", "median_o3", "median_so2", "median_no2", "median_pm10", "median_co"]:
for city_to_use in cities_list:
    print(city_to_use)

    train_data = CityDataP(SELECTED_COLUMN, "train", city_to_use)
    val_data = CityDataP(SELECTED_COLUMN, "test", city_to_use)
    sampleLoader = DataLoader(train_data, 32, shuffle=True, num_workers=4, worker_init_fn=seed_worker, generator=g)
    val_loader = DataLoader(val_data, 4096, shuffle=False, num_workers=4, worker_init_fn=seed_worker, generator=g)

    # RNN
    RNNmodel = RNN(1, INPUT_DIM+1, HIDDEN_DIM, LAYER_DIM).to(device)
    train(sampleLoader, val_loader, RNNmodel, "RNN", SELECTED_COLUMN)
    # LSTM
    LSTMmodel = LSTM(1, INPUT_DIM+1, HIDDEN_DIM, LAYER_DIM).to(device)
    train(sampleLoader, val_loader, LSTMmodel, "LSTM", SELECTED_COLUMN)
    # BiLSTM
    # BiLSTMmodel = LSTM(1, INPUT_DIM+1, HIDDEN_DIM, LAYER_DIM, bidirectional=True).to(device)
    # train(sampleLoader, val_loader, BiLSTMmodel, "BiLSTM", SELECTED_COLUMN)
    # TransLSTM
    TransLSTMmodel = TransLSTM(num_layers=3, D=16, H=5, hidden_mlp_dim=32, inp_features=9, out_features=1, dropout_rate=0.1, LSTM_module = LSTM(4, INPUT_DIM+1, HIDDEN_DIM, LAYER_DIM, bidirectional = False).to(device), attention_type='regular').to(device) # cosine_square, cosine, regular # 6L, 12H
    train(sampleLoader, val_loader, TransLSTMmodel, "TransLSTM", SELECTED_COLUMN, mask=True)
    # Transformer
    Transmodel = Transformer(num_layers=6, D=16, H=10, hidden_mlp_dim=32, inp_features=9, out_features=1, dropout_rate=0.1, attention_type='regular', SL=SEQ_LENGTH).to(device) # cosine_square, cosine, regular # 6L, 12H
    train(sampleLoader, val_loader, Transmodel, "Transformer", SELECTED_COLUMN, mask=True)
    # CosFormer
    TransCosModel = Transformer(num_layers=6, D=16, H=10, hidden_mlp_dim=32, inp_features=9, out_features=1, dropout_rate=0.1, attention_type='cosine', SL=SEQ_LENGTH).to(device) # cosine_square, cosine, regular # 6L, 12
    train(sampleLoader, val_loader, TransCosModel, "CosFormer", SELECTED_COLUMN, mask=True)
    # CosSquareFormer
    TransCosSquare = Transformer(num_layers=6, D=16, H=10, hidden_mlp_dim=32, inp_features=9, out_features=1, dropout_rate=0.1, attention_type='cosine_square', SL=SEQ_LENGTH).to(device) # cosine_square, cosine, regular # 6L, 12H
    train(sampleLoader, val_loader, TransCosModel, "CosSquareFormer", SELECTED_COLUMN, mask=True)


Albuquerque
set SoftDTW loss
Start RNN training


1.0495890104303176: 100%|██████████| 51/51 [00:08<00:00,  6.14it/s]


valid samples: 54.0
Eval MSE:  60.693155924479164
Eval RMSE: median_no2:  7.790581231492241
Eval MAPE: median_no2:  51.410173486780245


0.5767298896669172: 100%|██████████| 51/51 [00:07<00:00,  6.79it/s]


valid samples: 54.0
Eval MSE:  52.241984049479164
Eval RMSE: median_no2:  7.227861651240923
Eval MAPE: median_no2:  49.29856194390191


...

0.36040228503007515: 100%|██████████| 51/51 [00:07<00:00,  7.18it/s]


valid samples: 54.0
Eval MSE:  55.56585015190972
Eval RMSE: median_no2:  7.454250475528021
Eval MAPE: median_no2:  48.03757844147859


0.3759632072612351: 100%|██████████| 51/51 [00:07<00:00,  6.54it/s] 


valid samples: 54.0
Eval MSE:  55.42735460069444
Eval RMSE: median_no2:  7.44495497640479
Eval MAPE: median_no2:  48.35146798027886


0.33007418885827067:  57%|█████▋    | 29/51 [00:04<00:02,  7.93it/s]

In [ ]:
print("MSEs")
for key in MSEs :
    for val in MSEs[key]:
        print(key, val, end=",")
    print()
print()    

print("RMSEs")
for key in RMSEs :
    for val in RMSEs[key]:
        print(key, val, end=",")
    print()
print() 

print("MAPEss")
for key in MAPEs :
    for val in MAPEs[key]:
        print(key, val, end=",")
    print()
print() 

MSEs
RNN 1.5947319313331887,RNN 6.977589925130208,RNN 0.5268048679127413,RNN 17.087957311559606,RNN 9.462461683485243,RNN 7.422602900752315,RNN 54.4178955078125,RNN 18.58376510054977,RNN 23.37375217013889,RNN 20.76759885392099,RNN 1.2858943232783564,RNN 3.5167964719376474,RNN 6.070314760561343,RNN 2.5149047286422164,RNN 1.2347443192093461,RNN 2.351293917055483,RNN 6.973689326533565,RNN 2000.0,RNN 6.338933404886498,RNN 6.9910634358723955,RNN 7.222043718610491,RNN 2.851085804126881,RNN 2000.0,RNN 43.191677517361114,RNN 2000.0,RNN 25.28709129050926,RNN 2000.0,RNN 2000.0,RNN 14.012980143229166,RNN 2000.0,RNN 1.193588963261357,RNN 4.167152687355324,RNN 2000.0,RNN 12.494936342592593,RNN 15.694982458043981,RNN 6.596064814814815,RNN 1.1239132351345487,RNN 8.222720675998264,RNN 6.454164293077257,RNN 0.3453419296829789,RNN 0.5236163315949617,RNN 6.03844819245515,RNN 2000.0,RNN 1.291515209056713,RNN 8.236503883644387,RNN 2000.0,RNN 0.30189747280544704,RNN 13.00746324327257,RNN 8.654309873227719,R

In [ ]:
for city_to_use in cities_list:
    print(city_to_use)
    
    train_data = CityDataForecast(SELECTED_COLUMN, "train", city_to_use)
    val_data = CityDataForecast(SELECTED_COLUMN, "test", city_to_use)
    sampleLoader = DataLoader(train_data, 32, shuffle=True, num_workers=4, worker_init_fn=seed_worker, generator=g)
    val_loader = DataLoader(val_data, 4096, shuffle=False, num_workers=4, worker_init_fn=seed_worker, generator=g)

    OLS = LinearRegression(input_dim=9)
    train(sampleLoader, val_loader, OLS, "OLS", SELECTED_COLUMN, LUR=True)
    LASSO = LinearRegression(input_dim=9)
    train(sampleLoader, val_loader, LASSO, "LASSO", SELECTED_COLUMN, LUR=True, l1=True)
    Ridge = LinearRegression(input_dim=9)
    train(sampleLoader, val_loader, Ridge, "Ridge", SELECTED_COLUMN, LUR=True, l2=True)
    Elastic = LinearRegression(input_dim=9)
    train(sampleLoader, val_loader, Elastic, "Elastic", SELECTED_COLUMN, LUR=True, l1=True, l2=True)

Albuquerque
set l1,l2 loss
Start OLS training


0.39019584141928576: 100%|██████████| 29/29 [00:03<00:00,  7.92it/s]


valid samples: 60.0
Eval MSE:  6.169337463378906
Eval RMSE: median_no2:  2.4838151024943276
Eval MAPE: median_no2:  72.11670557657878


0.28152836037093193: 100%|██████████| 29/29 [00:03<00:00,  8.18it/s]


valid samples: 60.0
Eval MSE:  5.018760681152344
Eval RMSE: median_no2:  2.240259065633335
Eval MAPE: median_no2:  57.871513366699226


...

valid samples: 60.0
Eval MSE:  1.5965988159179687
Eval RMSE: median_no2:  1.2635659127714585
Eval MAPE: median_no2:  35.24433771769206


0.15936258679320073: 100%|██████████| 29/29 [00:02<00:00,  9.73it/s]


valid samples: 60.0
Eval MSE:  1.2886943817138672
Eval RMSE: median_no2:  1.1352067572534386
Eval MAPE: median_no2:  33.86786142985026


0.17086574846300587: 100%|██████████| 29/29 [00:03<00:00,  9.52it/s]


valid samples: 60.0
Eval MSE:  1.0276845296223958
Eval RMSE: median_no2:  1.0137477642995796
Eval MAPE: median_no2:  35.60832659403483
Elastic
Best MSE : 1.0204919179280598
RMSE : 1.0101940001445564
MAPE : 32.52997716267904



In [ ]:
print("MSEs")
for key in MSEs :
    for val in MSEs[key]:
        print(key, val, end=",")
    print()
print()    

print("RMSEs")
for key in RMSEs :
    for val in RMSEs[key]:
        print(key, val, end=",")
    print()
print() 

print("MAPEss")
for key in MAPEs :
    for val in MAPEs[key]:
        print(key, val, end=",")
    print()
print() 

MSEs
RNN 1.5947319313331887,RNN 6.977589925130208,RNN 0.5268048679127413,RNN 17.087957311559606,RNN 9.462461683485243,RNN 7.422602900752315,RNN 54.4178955078125,RNN 18.58376510054977,RNN 23.37375217013889,RNN 20.76759885392099,RNN 1.2858943232783564,RNN 3.5167964719376474,RNN 6.070314760561343,RNN 2.5149047286422164,RNN 1.2347443192093461,RNN 2.351293917055483,RNN 6.973689326533565,RNN 2000.0,RNN 6.338933404886498,RNN 6.9910634358723955,RNN 7.222043718610491,RNN 2.851085804126881,RNN 2000.0,RNN 43.191677517361114,RNN 2000.0,RNN 25.28709129050926,RNN 2000.0,RNN 2000.0,RNN 14.012980143229166,RNN 2000.0,RNN 1.193588963261357,RNN 4.167152687355324,RNN 2000.0,RNN 12.494936342592593,RNN 15.694982458043981,RNN 6.596064814814815,RNN 1.1239132351345487,RNN 8.222720675998264,RNN 6.454164293077257,RNN 0.3453419296829789,RNN 0.5236163315949617,RNN 6.03844819245515,RNN 2000.0,RNN 1.291515209056713,RNN 8.236503883644387,RNN 2000.0,RNN 0.30189747280544704,RNN 13.00746324327257,RNN 8.654309873227719,R